# Récupération des données pour l'étude communale

## 1- Importation des librairies externes et classiques

In [1]:
# Importation des librairies externes
!pip install git+https://github.com/InseeFrLab/Py-Insee-Data@master --ignore-install PyYAML
!pip install python-Levenshtein
!pip install openpyxl

  Cloning https://github.com/InseeFrLab/Py-Insee-Data (to revision master) to /tmp/pip-req-build-fcmvfqjh
  Running command git clone --filter=blob:none -q https://github.com/InseeFrLab/Py-Insee-Data /tmp/pip-req-build-fcmvfqjh
  Resolved https://github.com/InseeFrLab/Py-Insee-Data to commit 52139da402bc33da6b8892d7d3208f1d2ddfaa02
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     |████████████████████████████████| 661 kB 1.6 MB/s            
     |████████████████████████████████| 60 kB 15.1 MB/s            
     |████████████████████████████████| 11.5 MB 28.0 MB/s            
     |████████████████████████████████| 62 kB 1.9 MB/s             
     |████████████████████████████████| 76 kB 11.6 MB/s            
     |████████████████████████████████| 235 kB 22.2 MB/s            
     |████████████████████████████████| 503 kB 20.7 MB/s            
     |████████████████████████████████| 255 kB 23.7

In [2]:
# Importation des librairies 
import requests
import pandas as pd
import pynsee
import pynsee.download
from pynsee.sirene import *
import time
from tqdm import tqdm
from pynsee.utils.init_conn import init_conn
import os
from zipfile import ZipFile, Path
import io
import numpy as np

In [3]:
# token API insee
init_conn(insee_key=os.environ['INSEE_KEY'], insee_secret=os.environ['INSEE_SECRET'])

## 2- Téléchargement des données de SIRENE

In [4]:
### Téléchargement des données Sirene sur les établissements Siret de boucherie-charcuterie (code 4722Z) à l'instant T
# Téléchargement via Insee sirene data  by sirene.fr website

session = requests.session()
URL_ACCUEIL = "https://www.sirene.fr/sirene/public/creation-fichier"
URL_REQUETE_POST = "https://www.sirene.fr/sirene/public/verifier-creation-fichier.action"
URL_DEVIS="https://www.sirene.fr/sirene/client//sirene/client/devis-demande.action"

URL_RECAP="https://www.sirene.fr/sirene/client/telechargement-pret-fichier.action"
URL_TELECHARGEMENT_POST="https://www.sirene.fr/sirene/client/telechargement-fichier"
POST_DATA={
'fichier.caracteristiques.etat.actif': 'true',
'__checkbox_fichier.caracteristiques.etat.actif': 'true',
'__checkbox_fichier.caracteristiques.etat.ferme': 'true',
'fichier.caracteristiques.typeEtablissement': '1',
'fichier.caracteristiques.geoFile': 'false',
'fichier.caracteristiques.cjFile': 'false',
'fichier.caracteristiques.apeFile': 'false',
'fichier.caracteristiques.sirenSiretFile': 'false',
'fichiersCriteres.criteresGeographie.upload': '(binaire)',
'__multiselect_fichier.criteres.geographie.region.modalites': '',
'__multiselect_fichier.criteres.geographie.departement.modalites': '',
'fichiersCriteres.criteresActivite.upload': '(binaire)',
'__multiselect_fichier.criteres.activite.section.modalites': '',
'__multiselect_fichier.criteres.activite.division.modalites': '',
'__multiselect_fichier.criteres.activite.groupe.modalites': '',
'fichier.criteres.activite.sousClasse.modalites': '4722Z',
'__multiselect_fichier.criteres.activite.sousClasse.modalites': '',
'fichiersCriteres.criteresNature.upload': '(binaire)',
'__multiselect_fichier.criteres.nature.entrepriseNiveau1.modalites': '',
'__multiselect_fichier.criteres.nature.entrepriseNiveau2.modalites': '',
'__multiselect_fichier.criteres.nature.entrepriseNiveau3.modalites': '',
'__multiselect_fichier.criteres.trancheEffectif.modalites': '',
'fichier.criteres.dateCreation.date': '',
'fichier.criteres.dateCreation.dateFin': '',
'fichier.criteres.dateMaj.date': '',
'fichier.criteres.dateMaj.dateFin': '',
'fichiersCriteres.criteresSirenSiret.upload': '(binaire)',
'__multiselect_fichier.criteres.criteresCatEnt.modalites': '',
'strutsLocale': 'fr_FR',
}

result=session.get(URL_ACCUEIL)
result=session.post(URL_REQUETE_POST, data=POST_DATA)


result=session.get(URL_RECAP)

result=session.get(URL_DEVIS)

result=session.post(URL_TELECHARGEMENT_POST)

donnees_sirene_fr=pd.read_csv(io.StringIO(result.content.decode('utf-8')),dtype=str)


* les données de Sirene.fr sont des données d'entreprises France entière. Pour l'étude, le nombre d'établissement par commune est considéré :

In [5]:
donnees_sirene_fr_agregees_par_com = donnees_sirene_fr.groupby("codeCommuneEtablissement").size().reset_index(name='NbBoucheriesPaCom').set_index("codeCommuneEtablissement")
donnees_sirene_fr_agregees_par_com.describe()

,NbBoucheriesPaCom
count,7025.000000
mean,2.863630
std,6.460337
min,1.000000
25%,1.000000
50%,1.000000
75%,2.000000
max,179.000000


In [6]:
donnees_sirene_fr_agregees_par_com

,NbBoucheriesPaCom
codeCommuneEtablissement,
01004,7
01021,1
01024,1
01025,3
01026,1
...,...
97610,1
97611,4
97615,1


## 3- Téléchargement des données du ministère de l'intérieur

In [11]:
#Téléchargement des données de crimes et délits rapportés déclarés auprès des commissariats 
url_datagouv_min_interieur="https://www.data.gouv.fr/fr/datasets/r/d792092f-b1f7-4180-a367-d043200c1520"

donnees_police_nationale_2018=pd.read_excel(url_datagouv_min_interieur, sheet_name='Services PN 2018')
donnees_gendarmerie_nationale_2018=pd.read_excel(url_datagouv_min_interieur, sheet_name='Services GN 2018' )

del donnees_police_nationale_2018[donnees_police_nationale_2018.columns[0]]
del donnees_gendarmerie_nationale_2018[donnees_gendarmerie_nationale_2018.columns[0]]

url_localisation_pn="https://www.data.gouv.fr/fr/datasets/r/2cb2f356-42b2-4195-a35c-d4e4d986c62b"
url_localisation_gn="https://www.data.gouv.fr/fr/datasets/r/d6a43ef2-d302-4456-90e9-ff2c47cac562"
donnees_localisation_pn=pd.read_csv(url_localisation_pn,sep=';')
donnees_localisation_gn=pd.read_csv(url_localisation_gn,sep=';')

In [49]:
#donnees_gendarmerie_nationale_2018[donnees_gendarmerie_nationale_2018.columns[1:]]=donnees_gendarmerie_nationale_2018[donnees_gendarmerie_nationale_2018.columns[1:]].fillna(0).astype(int)
donnees_gendarmerie_nationale_2018

,Départements,01,01.1,01.2,01.3,02,02.1,02.2,02.3,02.4,...,974,974.1,974.2,978,987,987.1,988,988.1,988.2,988.3
0,Libellé index \ CGD,CGD BELLEY,CGD BOURG EN BRESSE,CGD GEX,CGD TREVOUX,CGD CHATEAU THIERRY NOGENTEL,CGD LAON,CGD SOISSONS,CGD ST QUENTIN,CGD VERVINS,...,CGD ST BENOIT,CGD ST PAUL,CGD ST PIERRE,CGD ST MARTIN ST BARTHELEMY,CGD LES ARCHIPELS PAPEETE,CGD LES ILES DU VENT FAAA,CGD KONE,CGD LA FOA,CGD NOUMEA,CGD POINDIMIE
1,Règlements de compte entre malfaireurs,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Homicides pour voler et à l'occasion de vols,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Homicides pour d'autres motifs,0,0,0,0,0,2,0,1,0,...,1,2,3,0,2,1,0,5,3,2
4,Tentatives d'homicides pour voler et à l'occas...,0,0,1,0,0,0,0,0,0,...,0,0,0,1,0,0,0,1,2,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
103,Infractions à l'exercice d'une profession règl...,0,4,1,1,0,2,0,0,1,...,1,3,4,1,0,0,1,0,1,2
104,Infractions au droit de l'urbanisme et de la c...,3,6,17,8,3,7,5,5,1,...,6,29,48,16,8,18,0,5,12,0
105,Fraudes fiscales,0,0,1,1,1,0,0,0,0,...,0,1,1,0,0,0,1,0,0,0
106,Autres délits économiques et financiers,0,2,5,1,3,3,1,0,0,...,6,6,16,2,3,3,0,1,1,0


## Traitement des données de la Police Nationale

In [14]:
donnees_police_nationale_2018_par_com=donnees_police_nationale_2018.T
donnees_police_nationale_2018_par_com.columns = donnees_police_nationale_2018_par_com.iloc[0]
donnees_police_nationale_2018_par_com=donnees_police_nationale_2018_par_com.iloc[1: , :]

donnees_police_nationale_2018_par_com['Localisation']=donnees_police_nationale_2018_par_com.index+" "+donnees_police_nationale_2018_par_com['Périmètres'] +" "+donnees_police_nationale_2018_par_com['Libellé index \ CSP']
donnees_police_nationale_2018_par_com['Localisation'].to_csv("toto.csv")
# donnees_police_nationale_2018_par_dep.drop(donnees_police_nationale_2018_par_dep[donnees_police_nationale_2018_par_dep.index.str[:2]=='98'].index,inplace=True)
# donnees_police_nationale_2018_par_dep["departement"]=np.where(donnees_police_nationale_2018_par_dep.index.str[:2]=='97', donnees_police_nationale_2018_par_dep.index.str[:3],donnees_police_nationale_2018_par_dep.index.str[:2])

# donnees_police_nationale_2018_par_dep = donnees_police_nationale_2018_par_dep.groupby("departement").sum() 
donnees_police_nationale_2018_par_com

Départements,Périmètres,Libellé index \ CSP,Règlements de compte entre malfaireurs,Homicides pour voler et à l'occasion de vols,Homicides pour d'autres motifs,Tentatives d'homicides pour voler et à l'occasion de vols,Tentatives homicides pour d'autres motifs,Coups et blessures volontaires suivis de mort,Autres coups et blessures volontaires criminels ou correctionnels,Prises d'otages à l'occasion de vols,...,Index non utilisé,Index non utilisé,"Prix illicittes, publicité fausse et infractions aux règles de la concurrence",Achats et ventes sans factures,Infractions à l'exercice d'une profession règlementée,Infractions au droit de l'urbanisme et de la construction,Fraudes fiscales,Autres délits économiques et financiers,Autres délits,Localisation
01,DCPAF,BMRA 01,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,2,01 DCPAF BMRA 01
01.1,DCPAF,SPAFT PREVESSIN,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,5,01.1 DCPAF SPAFT PREVESSIN
01.2,DCSP,CIAT DE BOURG EN BRESSE,0,0,2,0,1,1,236,0,...,0,0,0,0,1,1,0,3,223,01.2 DCSP CIAT DE BOURG EN BRESSE
01.3,DCSP,CIAT D'OYONNAX,0,0,1,0,0,0,135,0,...,0,0,0,0,1,1,0,0,64,01.3 DCSP CIAT D'OYONNAX
02,DCSP,CIAT DE CHATEAU THIERRY,0,0,2,0,0,0,102,0,...,0,0,0,0,0,0,0,0,47,02 DCSP CIAT DE CHATEAU THIERRY
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
987.1,DCPAF,SPAF TAHITI FAAA,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,987.1 DCPAF SPAF TAHITI FAAA
987.2,DCSP,CIAT DE PAPEETE,0,0,1,0,0,0,401,0,...,0,0,0,0,0,3,0,0,127,987.2 DCSP CIAT DE PAPEETE
988,DCPAF,DPAF UJI 988,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,2,988 DCPAF DPAF UJI 988
988.1,DCPAF,SPAF NOUMEA LA TONTOUTA,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,988.1 DCPAF SPAF NOUMEA LA TONTOUTA


* Agrégation par département, transposition et fusion :

In [ ]:
donnees_police_nationale_2018_par_dep = donnees_police_nationale_2018_par_dep.sum(axis=1).reset_index("departement").set_index("departement")
donnees_police_nationale_2018_par_dep.rename(columns={0: "Total des crimes et délits relevés par la Police Nationale"},inplace=True)

## Traitement des données de la Gendarmerie Nationale

In [ ]:
donnees_gendarmerie_nationale_2018_par_dep=donnees_gendarmerie_nationale_2018.T
donnees_gendarmerie_nationale_2018_par_dep.columns = donnees_gendarmerie_nationale_2018_par_dep.iloc[0]
donnees_gendarmerie_nationale_2018_par_dep=donnees_gendarmerie_nationale_2018_par_dep.iloc[1: , :]

donnees_gendarmerie_nationale_2018_par_dep.drop(donnees_gendarmerie_nationale_2018_par_dep[donnees_gendarmerie_nationale_2018_par_dep.index.str[:2]=='98'].index,inplace=True)
donnees_gendarmerie_nationale_2018_par_dep["departement"]=np.where(donnees_gendarmerie_nationale_2018_par_dep.index.str[:2]=='97', donnees_gendarmerie_nationale_2018_par_dep.index.str[:3],donnees_gendarmerie_nationale_2018_par_dep.index.str[:2])

donnees_gendarmerie_nationale_2018_par_dep = donnees_gendarmerie_nationale_2018_par_dep.groupby("departement").sum() 
donnees_gendarmerie_nationale_2018_par_dep

In [ ]:
donnees_gendarmerie_nationale_2018_par_dep = donnees_gendarmerie_nationale_2018_par_dep.sum(axis=1).reset_index("departement").set_index("departement")
donnees_gendarmerie_nationale_2018_par_dep.rename(columns={0: "Total des crimes et délits relevés par la Gendarmerie Nationale"},inplace=True)
donnees_gendarmerie_nationale_2018_par_dep

## Fusion des données de la Police Nationale et de la Gendarmerie Nationale 

In [ ]:
merged_crimes_delits_2018_par_dep = donnees_police_nationale_2018_par_dep.merge(donnees_gendarmerie_nationale_2018_par_dep, on="departement", how='left')
merged_crimes_delits_2018_par_dep

## 4 - Téléchargement des revenus localisés

In [ ]:
#Téléchargement des données Filosofi au niveau IRIS (millénisme 2018)
URL_FILO_DECL_2018="https://www.insee.fr/fr/statistiques/fichier/5055909/BASE_TD_FILO_DEC_IRIS_2018.xlsx"
URL_FILO_DISP_2018="https://www.insee.fr/fr/statistiques/fichier/5055909/BASE_TD_FILO_DISP_IRIS_2018.xlsx"

donnees_filo_decl_2018=pd.read_excel(URL_FILO_DECL_2018, sheet_name='IRIS_DEC', skiprows=5)
donnees_filo_disp_2018=pd.read_excel(URL_FILO_DISP_2018, sheet_name='IRIS_DISP', skiprows=5)

In [ ]:
URL_FILO_ZIP_2018="https://www.insee.fr/fr/statistiques/fichier/5009236/base-cc-filosofi-2018_CSV_geo2021.zip"
#Télécharger le zip de l'URL
r=requests.get(URL_FILO_ZIP_2018)

open("base-cc-filosofi-2018_CSV_geo2021.zip", 'wb').write(r.content)
    
    
with ZipFile("base-cc-filosofi-2018_CSV_geo2021.zip",'r') as myzip:
    data = myzip.open("cc_filosofi_2018_COM-geo2021.CSV")
    data2 = myzip.open("cc_filosofi_2018_DEP-geo2021.CSV")
    
donnees_filo_communes_2018=pd.read_csv(data,sep=';')       

In [ ]:
donnees_filo_communes_2018

In [ ]:
donnees_filo_departement_2018=donnees_filo_departement_2018.set_index("CODGEO")

In [ ]:
donnees_filo_departement_2018

## 5 - Téléchargement des populations légales

In [ ]:
#Téléchargement des populations légales du millénisme 2018 
URL_POP_LEG="https://www.insee.fr/fr/statistiques/fichier/4989724/ensemble.zip"

#Télécharger le zip de l'URL
r=requests.get(URL_POP_LEG)

open("ensemble.zip", 'wb').write(r.content)
    
    
with ZipFile("ensemble.zip",'r') as myzip:
    data = myzip.open("Departements.csv")

donnees_pop_leg_18=pd.read_csv(data,sep=';')     

In [ ]:
donnees_pop_leg_18.sample(10)

In [ ]:
donnees_pop_leg_18=donnees_pop_leg_18.set_index("CODDEP")

In [ ]:
donnees_pop_leg_18.describe()

## 6 - Téléchargement des données d'emplois localisées

In [ ]:
#Téléchargement des données d'emploi localisées
#donnees_emploi_statut_18 = pynsee.download.telechargerDonnees("ESTEL_T201", date = "2018")
#donnees_emploi_statut_18.sample(10)

#donnees_emploi_sexe_18 = pynsee.download.telechargerDonnees("ESTEL_T202", date = "2018")
#donnees_emploi_sexe_18.sample(10)


URL_TAUX_CHOMAGE_PAR_DEP="https://www.insee.fr/fr/statistiques/fichier/2012804/sl_etc_2021T2.xls"


donnees_chomage_par_departement_series_longues=pd.read_excel(URL_TAUX_CHOMAGE_PAR_DEP, sheet_name='Département', skiprows=3)

donnees_chomage_par_departement_series_longues=donnees_chomage_par_departement_series_longues.iloc[:100]
donnees_chomage_par_departement_series_longues=donnees_chomage_par_departement_series_longues.set_index("Code")
donnees_chomage_par_departement_series_longues.index=donnees_chomage_par_departement_series_longues.index.astype(str)

In [ ]:
donnees_chomage_par_departement_series_longues.describe()

## Telechargement des données de répartition de gendarmes et policiers en 2019 sur le territoire (au niveau départemental)

In [ ]:
URL_NB_PN_GN_DEP = "https://www.insee.fr/fr/statistiques/fichier/5763601/IREF_SECUR21-F29.xlsx"
donnees_nb_pn_gn_dep=pd.read_excel(URL_NB_PN_GN_DEP, sheet_name='Figure 3', skiprows=1)
donnees_nb_pn_gn_dep = donnees_nb_pn_gn_dep.set_index("Unnamed: 0")
donnees_nb_pn_gn_dep=donnees_nb_pn_gn_dep.iloc[:100]
donnees_nb_pn_gn_dep=donnees_nb_pn_gn_dep.rename(columns={"Unnamed: 0": "CODGEO", "Unnamed: 1": "Département", "Taux pour 10 000 habitants":"Nb_PN_GN_dep_100k_hab"}, index={'Unnamed:0': 'CODDEP'})
donnees_nb_pn_gn_dep.index = donnees_nb_pn_gn_dep.index.astype(str)
donnees_nb_pn_gn_dep.index.names = ['CODDEP']
donnees_nb_pn_gn_dep


## 7 - Téléchargement du Code Officiel Géographique

* Pour des raisons de simplicité, nous travaillons en géographie 2021 :

In [ ]:
#cog_commune = pynsee.download.telechargerDonnees("COG_COMMUNE", date = "dernier")
#URL_COG_2021="https://www.insee.fr/fr/statistiques/fichier/5057840/commune2021-csv.zip"

#Télécharger le zip de l'URL
#r=requests.get(URL_COG_2021)

#open("commune2021-csv.zip", 'wb').write(r.content)
    
    
#with ZipFile("commune2021-csv.zip",'r') as myzip:
#    data = myzip.open("commune2021.csv")

#donnees_cog_2021=pd.read_csv(data)     

In [ ]:
#donnees_cog_2021

## 8 - Fusion des données

In [ ]:
donnees_2018=merged_crimes_delits_2018_par_dep.merge(donnees_filo_departement_2018,left_index=True,right_index=True)
donnees_2018=donnees_2018.merge(donnees_pop_leg_18,left_index=True,right_index=True)
donnees_2018=donnees_2018.merge(donnees_sirene_fr_agregees_par_departement,left_index=True,right_index=True)
donnees_2018=donnees_2018.merge(donnees_chomage_par_departement_series_longues,left_index=True,right_index=True)
donnees_2018=donnees_2018.merge(donnees_nb_pn_gn_dep,left_index=True,right_index=True)
donnees_2018=donnees_2018[['REG', 'Libellé', 'Total des crimes et délits relevés par la Police Nationale', 'Total des crimes et délits relevés par la Gendarmerie Nationale', 'MED18' , 'TP6018', 'D118', 'D918', 'RD18', 'PTOT','T1_2018', 'NbBoucheriesParDep', 'Nb_PN_GN_dep_100k_hab']]
donnees_2018

* On se restreint au champ métropolitain car les données d'outre mer ne sont pas exhaustives sur toutes nos sources 

In [ ]:
donnees_2018 = donnees_2018[donnees_2018.index.str[:2]!='97']
donnees_2018
donnees_2018.describe()

* On fait l'hypothèse d'unicité du dépôt de plainte ou de signalement en gardarmerie ou bien en commissariat, ce qui permet de sommer les crimes et délits au niveau départemental

In [ ]:
donnees_2018['Total des crimes et délits relevés par la PN et GN'] = donnees_2018[['Total des crimes et délits relevés par la Police Nationale', 'Total des crimes et délits relevés par la Gendarmerie Nationale']].sum(axis=1)

In [ ]:
donnees_2018 = donnees_2018.rename(columns={"Total des crimes et délits relevés par la Police Nationale": "Crim_Del_PN", "Total des crimes et délits relevés par la Gendarmerie Nationale": "Crim_Del_GN", "Total des crimes et délits relevés par la PN et GN": "Crim_Del_PN_GN"})
donnees_2018

## 9 - On créé une base de données rapportant le nombre de crimes et délits, et le nombre de boucheries pour 100 000 habitants 


In [ ]:
donnees_2018_hab = donnees_2018
donnees_2018_hab['Nb_Boucherie_dep_hab']=round(donnees_2018_hab['NbBoucheriesParDep']/donnees_2018_hab['PTOT']*100000)
donnees_2018_hab['Crim_Del_GN_hab']=round(donnees_2018_hab['Crim_Del_GN']/donnees_2018_hab['PTOT']*100000)
donnees_2018_hab['Crim_Del_PN_hab']=round(donnees_2018_hab['Crim_Del_PN']/donnees_2018_hab['PTOT']*100000)
donnees_2018_hab['Crim_Del_PN_GN_hab']=round(donnees_2018_hab['Crim_Del_PN_GN']/donnees_2018_hab['PTOT']*100000)
donnees_2018_hab.drop(['Crim_Del_PN', 'Crim_Del_GN', 'PTOT', 'NbBoucheriesParDep'], axis=1, inplace = True)
donnees_2018_hab


In [ ]:
%store donnees_2018
%store donnees_2018_hab